# AutoGen but what if it could replicate itself?

___________
```
def research(query):
    ass_config = {}
    
    ass = AssistantAgent()
    upa = UserProxyAgent()
    upa.init_chat(with ass, query)
    
    return upa.last_message
```

deploy_agents() - provided to the
agent instead
___________
```
def deploy_agents(sys_msgs):
    def func(arg):
        ass_config = {}
        
        ass = AssistantAgent()
        upa = UserProxyAgent()
        upa.init_chat(with ass)
        
        return upa.last_message
    
    return research

```

In [6]:
import openai
import autogen
from autogen import (
    AssistantAgent, 
    ConversableAgent,
    UserProxyAgent, 
    config_list_from_json,
    Completion,  # openai.Completion wrapper || for single-prediction completions e.g. .create(**) with prompt='hi'
    ChatCompletion,  # openai.Completion wrapper || for Chats e.g. .create(**) with messages=list({'role':'user', 'content':'hi'})  https://microsoft.github.io/autogen/blog/
)

from dotenv import load_dotenv
import os, sys, glob

# Get API key
load_dotenv()
config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST")
openai.api_key = os.getenv("OPENAI_API_KEY")

Completion.clear_cache()
ChatCompletion.clear_cache()

In [4]:
'''
Look at this function.

It takes an input, intitates a 2-agent chat between:

a UserProxyAgent <-> an AssistantAgent
+ some functions that are useful

And returns the last message the UserProxyAgent received.

This is sick.
What if we can make this a recursive function?
What if we make an LLM construct this function when it needs one?
What limits does this function have?
When should it be used specifically?

- one test case is this function itself. I.e. let's let an LLM construct this function from an abstract function call.

```python
def symbolic_function(input):
    upa = autogen.UserProxyAgent()
    aa = autogen.AssistantAgent()
    upa.initiate_chat(aa, message=input)

    return upa.last_message()["content"]
```

'''
def research(query):
    llm_config_researcher = {
        "functions": [
            {
                "name": "search",
                "description": "google search for relevant information",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "Google search query",
                        }
                    },
                    "required": ["query"],
                },
            },
            {
                "name": "scrape",
                "description": "Scraping website content based on url",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "url": {
                            "type": "string",
                            "description": "Website url to scrape",
                        }
                    },
                    "required": ["url"],
                },
            },
        ],
        "config_list": config_list}

    researcher = autogen.AssistantAgent(
        name="researcher",
        system_message="Research about a given query, collect as many information as possible, and generate detailed research results with loads of technique details with all reference links attached; Add TERMINATE to the end of the research report;",
        llm_config=llm_config_researcher,
    )

    user_proxy = autogen.UserProxyAgent(
        name="User_proxy",
        code_execution_config={"last_n_messages": 2, "work_dir": "coding"},
        is_termination_msg=lambda x: x.get("content", "") and x.get(
            "content", "").rstrip().endswith("TERMINATE"),
        human_input_mode="TERMINATE",
        function_map={
            "search": search,
            "scrape": scrape,
        }
    )

    user_proxy.initiate_chat(researcher, message=query)

    # set the receiver to be researcher, and get a summary of the research report
    user_proxy.stop_reply_at_receive(researcher)
    user_proxy.send(
        "Give me the research report that just generated again, return ONLY the report & reference links", researcher)

    # return the last message the expert received
    return user_proxy.last_message()["content"]

In [5]:
# # Scrapped ideas for the function below

# # An appropriate system message provided in this example would be:
# # You are an expert researcher on a particular topic. Research about a given query, collect as much information as possible, 
# # and generate detailed research results with loads of technique details with all 
# # reference links attacheds

# FUNCTION_DESCRIPTION = """Deploy a pair of workers that can carry out a task together. 
# One worker is an assistant which carries out the task, and the other is a user proxy which monitors the assistant. 
# The workers will then carry out the task and return the result to you.
# This can be useful if you're carrying out a task to be done external the the one you are currently working on.
# """

# SYSTEM_MESSAGE_DESCRIPTION = """You must provide a system message that will help the agents understand their goals and provide relevant context.
# Start with a goal; provide ALL the relevant context; insist clarity and precision;
# """
# INITIAL_MESSAGE_DESCRIPTION = """You must specify an initial message that will initiate the conversation between the user proxy and the assistant.
# Provide all relevant context to the task being solved.
# """
# NAME_DESCRIPTION = "The name for the assistant worker."

In [2]:
'''
Now we don't really care about the details of the exact research function etc.

But let's develop a function that can generate this research function for us without defining it explicitly.

- How do tell the LLM that it can do this?
- Will the LLM know when it needs additional resources?
- The LLM will need to generate 
'''

FUNCTION_DESCRIPTION = """A worker that can carry out a well-defined task. 
The worker will then carry out the task and return the result to you.
The task requested should be a single confined task and have a clear end goal.
This can be useful if you're carrying out a task to be done external the the one you are currently working on.
"""

SYSTEM_MESSAGE_DESCRIPTION = """You must provide a message that will help the worker understand their goals and provide relevant context.
Start with a goal; provide ALL the relevant context; insist clarity and precision; make sure the worker knows when to stop and when it has completed its goal;
"""
INITIAL_MESSAGE_DESCRIPTION = """You must specify an initial message  that 
Provide all relevant context to the task being solved.
"""
NAME_DESCRIPTION = "The name for the assistant worker."
FUNCTION_JSON = {
    "name": "deploy_agents",
    "description": FUNCTION_DESCRIPTION,
    "parameters": {
        "type": "object",
        "properties": {
            "system_message": {
                "type": "string",
                "description": SYSTEM_MESSAGE_DESCRIPTION,
            },
            "initial_message": {
                "type": "string",
                "description": INITIAL_MESSAGE_DESCRIPTION,
            },
            # "name": {
            #     "type": "string",
            #     "description": NAME_DESCRIPTION,
            # },   # may overwhelm the LLM?
        },
        "required": ["system_message", "initial_message"],
    },
}
SYSTEM_MESSAGE_SUFFIX = """; Add TERMINATE to the end of your message;"""


def deploy_agents(
    system_message, 
    initial_message, 
    endpoint_message="Give me that report and / or code that you just made again, return ONLY the relevant code or information;",
    name='assistant'
):

    '''
    A function that deploys a 2-agent pair. Recursively defined.

    Assistant Agent <-- system_message, initial_message, name
    UserProxyAgent <-- predefined init

    returns the end of convo message.
    '''

    def optimize_system_message():
        """# TODO: implement an LLM that does this optimization;
        may not work as well as erroneous context may be generated;
        may return better if the LLM is trained / provided examples to do formatting better"""
        
        return system_message
    system_message = optimize_system_message()
    system_message += SYSTEM_MESSAGE_SUFFIX


    assistant = autogen.AssistantAgent(
        name=name,
        system_message=system_message,
        llm_config={
        "functions": [FUNCTION_JSON], # PROVIDE FUNCTION RECURSIVELY HERE! 
        "config_list": config_list},  
    )

    user_proxy = autogen.UserProxyAgent(
        name="User_proxy",
        code_execution_config={"last_n_messages": 2, "work_dir": "coding"},
        is_termination_msg=lambda x: x.get("content", "") and x.get(
            "content", "").rstrip().endswith("TERMINATE"),
        human_input_mode="ALWAYS",
        function_map={
            "deploy_agents": deploy_agents,
        }  # AND HERE!
    )

    # liftoff!
    user_proxy.initiate_chat(assistant, message=initial_message)

    # set the receiver to be assistant, and get a summary / more concrete message.
    user_proxy.stop_reply_at_receive(assistant)
    user_proxy.send(
        endpoint_message, assistant)

    # return the last message the expert received
    return user_proxy.last_message()["content"]


In [3]:
def init():
    ass0 = autogen.AssistantAgent(
        name='assistant',
        system_message="You are a lead project manager; You put a plan together to solve a task effectively, and delegate the task to a team of workers",
        llm_config={
            "functions": [FUNCTION_JSON],
            "config_list": config_list},  
    )

    upa0 = autogen.UserProxyAgent(
        name="User_proxy",
        code_execution_config={"last_n_messages": 2, "work_dir": "coding"},
        is_termination_msg=lambda x: x.get("content", "") and x.get(
            "content", "").rstrip().endswith("TERMINATE"),
        human_input_mode="ALWAYS",
        function_map={
            "deploy_agents": deploy_agents,
        }
    )
    # liftoff!
    upa0.initiate_chat(ass0, message='Design a plan to build the game snake in python')

In [7]:
init()

User_proxy (to assistant):

Design a plan to build the game snake in python

--------------------------------------------------------------------------------
assistant (to User_proxy):

Sure, let's break down the task of creating the game Snake in Python in phases:

Phase 1: Setup
- Install required Python packages.
- Create environment for coding.

Phase 2: Snake 
- Create a function that initializes game / snake variables.
- Create a function to take care of the game logic.

Phase 3: Food
- Create a function to initialize food.
- Create a function to place new food on the screen at a random place.

Phase 4: Display
- Create a function to update the display.
- Create a function to print the scores.

Phase 5: User Input
- Add operations for the game to accept user input and change the direction of the snake accordingly.

Phase 6: Game Over
- Add a function to check if the game is over.

Phase 7: Game Loop
- Create the main game loop which controls the game.

Phase 8: Testing
- Thorough

KeyboardInterrupt: 